## Сверточные нейронные сети для анализа текста

Задание
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек
2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [1]:
# Загрузка данных Google Colab

# from google.colab import files
# files.upload()

In [2]:
# pip install --upgrade xlrd

In [3]:
# pip install pymorphy2

In [4]:
# pip install stop-words

In [5]:
# Загрузка данных

import pandas as pd

df = pd.read_excel('./отзывы за лето.xls')

df.head(5)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [6]:
# Предобработка текста

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [7]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("не\s", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df['Content'] = df['Content'].apply(preprocess_text)

In [8]:
# Разделение выборки на тренировочную, тестовую и валидационную

from sklearn.model_selection import train_test_split

train_df, test_val_df = train_test_split(df, test_size=0.3, random_state=17)

test_df, val_df = train_test_split(test_val_df, test_size=0.5, random_state=17)

In [9]:
# Создание корпуса

train_corpus = " ".join(train_df['Content'])
train_corpus = train_corpus.lower()

In [10]:
# Токенизация корпуса

import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
# Настраиваемые параметры сети

max_words = 1000
max_len = 100
num_classes = 6

In [12]:
# Настраиваемые параметры обучения

epochs = 20
batch_size = 512
print_batch_n = 100

In [13]:
# Сортировка и отбор самых частотных токенов

from nltk.probability import FreqDist

tokens_filtered = [word for word in tokens if word.isalnum()]

dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [14]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [15]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [16]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in train_df['Content']], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in test_df['Content']], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in val_df['Content']], dtype=np.int32)

In [17]:
# Импорт библиотек для работы нейронной сети

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
# from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  
from keras.utils import np_utils

In [18]:
y_train = keras.utils.np_utils.to_categorical(train_df['Rating'], num_classes)
y_test = keras.utils.np_utils.to_categorical(test_df['Rating'], num_classes)
y_val = keras.utils.np_utils.to_categorical(val_df['Rating'], num_classes)

In [19]:
# Сеть

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [20]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [21]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 10s 363ms/step - loss: 1.5417 - accuracy: 0.6761 - val_loss: 1.2005 - val_accuracy: 0.6814
Epoch 2/20
26/26 [==============================] - 9s 333ms/step - loss: 1.0443 - accuracy: 0.7078 - val_loss: 0.9791 - val_accuracy: 0.6814


In [22]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 1s 78ms/step - loss: 0.9228 - accuracy: 0.7102


Test score: 0.9228094220161438
Test accuracy: 0.7102290987968445


In [23]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

7/7 [==============================] - 1s 78ms/step


Пробовала увеличить количество сверточных слоев - качество только ухудшилось

### Embedding слой с предобученными векторами

In [24]:
# Загрузка предобученной модели

!wget https://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz

!gunzip taiga_upos_skipgram_300_2_2018.vec.gz

--2022-06-06 13:59:23--  https://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346385366 (330M) [application/x-gzip]
Saving to: ‘taiga_upos_skipgram_300_2_2018.vec.gz’

taiga_upos_skipgram 100%[===================>] 330.34M  18.8MB/s    in 18s     

2022-06-06 13:59:42 (18.0 MB/s) - ‘taiga_upos_skipgram_300_2_2018.vec.gz’ saved [346385366/346385366]

gzip: taiga_upos_skipgram_300_2_2018.vec already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [25]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['Content'])
vocab_size = len(tokenizer.word_index) + 1

In [26]:
embeddings_index = {}
with open('taiga_upos_skipgram_300_2_2018.vec') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        word = word.split('_')[0]
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 209666 word vectors.


In [27]:
# Матрица эмбеддингов

embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 4886 words (4993 misses)


In [28]:
# Сеть

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=300, input_length=max_len, weights=[embedding_matrix],
    trainable=False))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [29]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [30]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 14s 513ms/step - loss: 1.5540 - accuracy: 0.6539 - val_loss: 1.3132 - val_accuracy: 0.6814
Epoch 2/20
26/26 [==============================] - 13s 500ms/step - loss: 1.0815 - accuracy: 0.7117 - val_loss: 0.9922 - val_accuracy: 0.7077


In [31]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 1s 168ms/step - loss: 0.9340 - accuracy: 0.7354


Test score: 0.9339936375617981
Test accuracy: 0.735398530960083


In [32]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

7/7 [==============================] - 1s 175ms/step


Качество модели выросло незначительно